# 1. 初始化
## 1.1 导入包

In [1]:
import csv
import json
from datetime import date, datetime
import datetime
from xgboost import XGBClassifier
import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


c:\Python39\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## 1.2 读入文件

存放在 list 中

### 1.2.1 训练集

In [2]:
TRAIN_l_fqdn = []
with open('5_question_train/fqdn.csv', 'r', encoding='utf-8') as f:
    f_fqdn = csv.reader(f)
    headers = next(f_fqdn)
    for row in f_fqdn:
        TRAIN_l_fqdn.append(row)

TRAIN_l_ip = {}
with open('5_question_train/ip.csv', 'r', encoding='utf-8') as f:
    f_ip = csv.reader(f)
    headers = next(f_ip)
    for row in f_ip:
        TRAIN_l_ip[row[0]] = row[1:]

# l_ipv6 = []
with open('5_question_train/ipv6.csv', 'r', encoding='utf-8') as f:
    f_ipv6 = csv.reader(f)
    headers = next(f_ipv6)
    for row in f_ipv6:
        # l_ipv6[row[0]] = row[1:]
        TRAIN_l_ip[row[0]] = row[1:]

TRAIN_l_access = []
with open('5_question_train/access.csv', 'r', encoding='utf-8') as f:
    f_access = csv.reader(f)
    headers = next(f_access)
    for row in f_access:
        TRAIN_l_access.append(row)

TRAIN_l_flint = []
with open('5_question_train/flint.csv', 'r', encoding='utf-8') as f:
    f_flint = csv.reader(f)
    headers = next(f_flint)
    for row in f_flint:
        TRAIN_l_flint.append(row)

TRAIN_l_label = []
with open('5_question_train/label.csv', 'r', encoding='utf-8') as f:
    f_label = csv.reader(f)
    headers = next(f_label)
    for row in f_label:
        TRAIN_l_label.append(row)


### 1.2.2 测试集

In [3]:
TEST_l_fqdn = []
with open('5_question_test/fqdn.csv', 'r', encoding='utf-8') as f:
    f_fqdn = csv.reader(f)
    headers = next(f_fqdn)
    for row in f_fqdn:
        TEST_l_fqdn.append(row)

TEST_l_ip = {}
with open('5_question_test/ip.csv', 'r', encoding='utf-8') as f:
    f_ip = csv.reader(f)
    headers = next(f_ip)
    for row in f_ip:
        TEST_l_ip[row[0]] = row[1:]

# l_ipv6 = []
with open('5_question_test/ipv6.csv', 'r', encoding='utf-8') as f:
    f_ipv6 = csv.reader(f)
    headers = next(f_ipv6)
    for row in f_ipv6:
        # l_ipv6[row[0]] = row[1:]
        TEST_l_ip[row[0]] = row[1:]

TEST_l_access = []
with open('5_question_test/access.csv', 'r', encoding='utf-8') as f:
    f_access = csv.reader(f)
    headers = next(f_access)
    for row in f_access:
        TEST_l_access.append(row)

TEST_l_flint = []
with open('5_question_test/flint.csv', 'r', encoding='utf-8') as f:
    f_flint = csv.reader(f)
    headers = next(f_flint)
    for row in f_flint:
        TEST_l_flint.append(row)


# 2 训练集特征提取

## 2.1 access

将 access 与 ip 一并处理。选取的特征有

- 访问次数
- 连续访问次数
- 访问 IP 数
- 按小时统计访问次数
- 按日期统计访问次数

In [4]:
# Initialization

tot_fqdn = 20512 # fqdn 编号总数

l_accesscount = [0] * tot_fqdn # 访问次数
l_multiaccesscount = [0] * tot_fqdn # 多次访问次数
# l_multiaccessip = [set() for _ in range(tot_fqdn)] # 多次访问IP
l_accesstime = [[0] * 24 for _ in range(tot_fqdn)] # 按小时统计访问次数
l_accessdate = [[0] * 92 for _ in range(tot_fqdn)] # 按日期统计访问次数
l_accessip = [0] * tot_fqdn  # 访问IP数

date2 = date(2020, 3, 1) # 开始日期

# count

for access in TRAIN_l_access:
    num = int(access[0][5:]) # 域名编号
    access_ip = int(access[1]) # 访问IP数
    access_count = int(access[2]) # 访问次数
    access_date = access[4] # 访问日期
    access_hour = int(access[5]) # 访问时间

    l_accesscount[num] += access_count # 编号为 num 的域名的访问次数
    if access_count > 1:
        l_multiaccesscount[num] += access_count  # 编号为 num 的域名的连续访问次数
        # l_multiaccessip[num].add(access_ip) # 编号为 num 的域名的连续访问 IP

    l_accesstime[num][access_hour] += access_count  # 编号为 num 的域名按小时统计时间段的访问次数

    date1 = datetime.datetime.strptime(access_date, '%Y%m%d').date()
    l_accessdate[num][(date1 - date2).days] += access_count  # 编号为 num 的域名按日期统计访问次数

    l_accessip[num] += access_ip  # 编号为 num 的域名的访问 IP 数


## 2.2 flint

选取的特征有

- 解析次数
- 按日期统计解析次数

In [5]:
l_notip = [] # 解析结果不是 IP 的域名

l_flintcount = [0] * tot_fqdn # 解析次数
l_flintdate = [[0] * 92 for _ in range(tot_fqdn)] # 按日期统计解析次数

for flint in TRAIN_l_flint:
    num = int(flint[0][5:])  # 域名编号
    typenum = int(flint[1])  # 域名类型
    if flint[2][0:4] == 'fqdn':
        refernum = int(flint[2][5:]) # 域名引用编号
        count = int(flint[3]) # 域名引用访问次数
        date = datetime.datetime.strptime(flint[4], '%Y%m%d').date() # 域名引用访问时间
        l_notip.append([num, typenum, refernum, count, date])
    else:
        ip = flint[2]  # 域名 IP
        count = int(flint[3])  # 域名访问次数
        date = datetime.datetime.strptime(flint[4], '%Y%m%d').date()  # 域名访问时间
        
    
    l_flintcount[num] += count # 编号为 num 的域名的解析次数
    l_flintdate[num][(date - date2).days] += count # 编号为 num 的域名按日期统计解析次数


## 2.3 label

提取标签，并为拆分训练集、测试集做准备。

In [6]:
Label_all = [-1] * tot_fqdn # 域名标签

for label in TRAIN_l_label:
    num = int(label[0][5:])  # 域名编号
    Label_all[num] = int(label[1])  # 域名标签


## 2.4 fqdn

进一步处理 2.1 - 2.3 得到的内容。

提取 fqdn 中的特征：

- 字符个数
- 数字个数
- 普通字符个数
- 特殊字符个数
- 单词字母个数
- 深度

并将所有内容整合。

In [7]:
TRAIN_num =[] # 训练集域名编号
TRAIN_feature = []  # 合并信息
TRAIN_label = []  # 域名标签

for fqdn in TRAIN_l_fqdn:
    url = fqdn[0] # 域名
    num = int(fqdn[1][5:]) # 域名编号

    n_character = 0 # 字符个数
    n_digit = 0 # 数字个数
    n_normal = 0 # 普通字符个数
    n_special = 0 # 特殊字符个数
    n_alpha = 0 # 单词字母个数
    n_depth = 0 # 深度
    
    n_access = l_accesscount[num] # 访问次数
    n_multiaccess = l_multiaccesscount[num] # 连续访问次数
    n_access_ip = l_accessip[num] # 访问 IP 数
    n_access_byhour = l_accesstime[num]  # 按小时统计访问次数
    n_access_bydate = l_accessdate[num]  # 按日期统计访问次数

    n_flint = l_flintcount[num] # 解析次数
    n_flint_bydate = l_flintdate[num] # 按日期统计解析次数

    is_normal_suffix = 0 # 是否是普通后缀
    commonRootList = ['cn', 'com', 'net', 'org', 'gov', 'info', 'edu']
    if url.split('.')[-1] in commonRootList:
        is_normal_suffix = 1

    is_alpha = False # 是否为单词字母

    for i in range(len(url)):
        if url[i] == '[':
            is_alpha = True
        elif url[i] == ']':
            is_alpha = False
        else:
            n_character += 1
            
            if url[i].isalnum():
                n_normal += 1
            else:
                n_special += 1

            if url[i].isdigit():
                n_digit += 1

            if is_alpha:
                n_alpha += 1
                
            if url[i] == '.':
                n_depth += 1
    
    tmp = [n_character, n_digit, n_normal, n_special, n_alpha, n_depth, is_normal_suffix,
           n_access, n_multiaccess, n_access_ip,
           n_flint]
    tmp.extend(n_access_byhour)
    tmp.extend(n_access_bydate)
    tmp.extend(n_flint_bydate)
    if Label_all[num] != -1:
        TRAIN_num.append(num)
        TRAIN_feature.append(tmp)
        TRAIN_label.append(Label_all[num])


# 3 测试集特征提取

## 3.1 access

In [8]:
# Initialization

tot_fqdn = 20512  # fqdn 编号总数

l_accesscount = [0] * tot_fqdn  # 访问次数
l_multiaccesscount = [0] * tot_fqdn  # 多次访问次数
# l_multiaccessip = [set() for _ in range(tot_fqdn)] # 多次访问IP
l_accesstime = [[0] * 24 for _ in range(tot_fqdn)]  # 按小时统计访问次数
l_accessdate = [[0] * 92 for _ in range(tot_fqdn)]  # 按日期统计访问次数
l_accessip = [0] * tot_fqdn  # 访问IP数

# count

for access in TEST_l_access:
    num = int(access[0][5:])  # 域名编号
    access_ip = int(access[1])  # 访问IP数
    access_count = int(access[2])  # 访问次数
    access_date = access[4]  # 访问日期
    access_hour = int(access[5])  # 访问时间

    l_accesscount[num] += access_count  # 编号为 num 的域名的访问次数
    if access_count > 1:
        l_multiaccesscount[num] += access_count  # 编号为 num 的域名的连续访问次数
        # l_multiaccessip[num].add(access_ip) # 编号为 num 的域名的连续访问 IP

    l_accesstime[num][access_hour] += access_count  # 编号为 num 的域名按小时统计时间段的访问次数

    date1 = datetime.datetime.strptime(access_date, '%Y%m%d').date()
    # 编号为 num 的域名按日期统计访问次数
    l_accessdate[num][(date1 - date2).days] += access_count

    l_accessip[num] += access_ip  # 编号为 num 的域名的访问 IP 数


## 3.2 flint

In [9]:
l_notip = []  # 解析结果不是 IP 的域名

l_flintcount = [0] * tot_fqdn  # 解析次数
l_flintdate = [[0] * 92 for _ in range(tot_fqdn)]  # 按日期统计解析次数

for flint in TEST_l_flint:
    num = int(flint[0][5:])  # 域名编号
    typenum = int(flint[1])  # 域名类型
    if flint[2][0:4] == 'fqdn':
        refernum = int(flint[2][5:])  # 域名引用编号
        count = int(flint[3])  # 域名引用访问次数
        date = datetime.datetime.strptime(
            flint[4], '%Y%m%d').date()  # 域名引用访问时间
        l_notip.append([num, typenum, refernum, count, date])
    else:
        ip = flint[2]  # 域名 IP
        count = int(flint[3])  # 域名访问次数
        date = datetime.datetime.strptime(flint[4], '%Y%m%d').date()  # 域名访问时间

    l_flintcount[num] += count  # 编号为 num 的域名的解析次数
    l_flintdate[num][(date - date2).days] += count  # 编号为 num 的域名按日期统计解析次数


## 3.3 fqdn

In [10]:
TEST_num =[] # 训练集域名编号
TEST_feature = []  # 合并信息
TEST_label = []  # 域名标签

for fqdn in TEST_l_fqdn:
    url = fqdn[0] # 域名
    num = int(fqdn[1][5:]) # 域名编号

    n_character = 0 # 字符个数
    n_digit = 0 # 数字个数
    n_normal = 0 # 普通字符个数
    n_special = 0 # 特殊字符个数
    n_alpha = 0 # 单词字母个数
    n_depth = 0 # 深度
    
    n_access = l_accesscount[num] # 访问次数
    n_multiaccess = l_multiaccesscount[num] # 连续访问次数
    n_access_ip = l_accessip[num] # 访问 IP 数
    n_access_byhour = l_accesstime[num]  # 按小时统计访问次数
    n_access_bydate = l_accessdate[num]  # 按日期统计访问次数

    n_flint = l_flintcount[num] # 解析次数
    n_flint_bydate = l_flintdate[num] # 按日期统计解析次数

    is_normal_suffix = 0 # 是否是普通后缀
    commonRootList = ['cn', 'com', 'net', 'org', 'gov', 'info', 'edu']
    if url.split('.')[-1] in commonRootList:
        is_normal_suffix = 1

    is_alpha = False # 是否为单词字母

    for i in range(len(url)):
        if url[i] == '[':
            is_alpha = True
        elif url[i] == ']':
            is_alpha = False
        else:
            n_character += 1
            
            if url[i].isalnum():
                n_normal += 1
            else:
                n_special += 1

            if url[i].isdigit():
                n_digit += 1

            if is_alpha:
                n_alpha += 1
                
            if url[i] == '.':
                n_depth += 1
    
    tmp = [n_character, n_digit, n_normal, n_special, n_alpha, n_depth, is_normal_suffix,
           n_access, n_multiaccess, n_access_ip,
           n_flint]
    tmp.extend(n_access_byhour)
    tmp.extend(n_access_bydate)
    tmp.extend(n_flint_bydate)
    TEST_num.append(num)
    TEST_feature.append(tmp)


# 4 训练与预测

In [22]:
params = { # 玄学调参
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'gamma': 0.1,
    'max_depth': 6,
    'lambda': 2,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 1,
    'eta': 0.05,
    'eval_metric': 'logloss'
}

# 构造训练集
dtrain = xgb.DMatrix(TRAIN_feature, TRAIN_label)
num_rounds = 5000
# xgboost模型训练
model = xgb.train(params, dtrain, num_rounds)

# 对测试集进行预测
dtest = xgb.DMatrix(TEST_feature)
TEST_label = model.predict(dtest)


[14:47:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [23]:
# 输出结果
with open("result2.csv", "w") as csvfile:
    csvfile.write("fqdn_no,label\n")
    for i in range(len(TEST_num)):
        csvfile.write("fqdn_" + str(TEST_num[i]) + "," + str(round(TEST_label[i])) + "\n")
